In [3]:
#Import library
import pandas as pd

#Set filepaths
schools_csv = "Resources/schools_complete.csv"
students_csv = "Resources/students_complete.csv"

#Read filepaths into pandas
schools_pd = pd.read_csv(schools_csv)
students_pd = pd.read_csv(students_csv)

In [4]:
#Merge dataframes
combined_pd = pd.merge(schools_pd,students_pd, how = "right", on = ["school_name"])

#Assess Data
combined_pd.head()
combined_pd.tail()
combined_pd.info()
combined_pd.dtypes
combined_pd.columns

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39170 entries, 0 to 39169
Data columns (total 11 columns):
School ID        39170 non-null int64
school_name      39170 non-null object
type             39170 non-null object
size             39170 non-null int64
budget           39170 non-null int64
Student ID       39170 non-null int64
student_name     39170 non-null object
gender           39170 non-null object
grade            39170 non-null object
reading_score    39170 non-null int64
math_score       39170 non-null int64
dtypes: int64(6), object(5)
memory usage: 3.6+ MB


Index(['School ID', 'school_name', 'type', 'size', 'budget', 'Student ID',
       'student_name', 'gender', 'grade', 'reading_score', 'math_score'],
      dtype='object')

In [5]:
#District Summary
columns = combined_pd.columns

total_schools = len(combined_pd["school_name"].unique())

total_students = len(combined_pd["student_name"])

total_budget = (combined_pd["budget"].unique()).sum()

avg_math_score = combined_pd["math_score"].mean()

avg_reading_score = combined_pd["reading_score"].mean()

pass_math_pd = combined_pd.loc[combined_pd["math_score"] >= 70, columns]
percent_pass_math = (len(pass_math_pd["math_score"])/len(combined_pd["math_score"]))*100

pass_read_pd = combined_pd.loc[combined_pd["reading_score"] >= 70, columns]
percent_pass_read = (len(pass_read_pd["reading_score"])/len(combined_pd["reading_score"]))*100


overall_pass_rate = ((percent_pass_math + percent_pass_read)/2)

district_summary_df = pd.DataFrame({"Total Schools": [total_schools],
                                 "Total Students": [total_students],
                                 "Total Budget": [total_budget],
                                 "Average Math Score": [avg_math_score],
                                 "Average Reading Score": [avg_reading_score],
                                 "% Passing Math": [percent_pass_math],
                                 "% Passing Reading": [percent_pass_read],
                                    "% Overall Pass Rate": [overall_pass_rate]
                                })
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Pass Rate
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.393158


In [6]:
#School Summary
combined_pd["% Pass Math"] = (combined_pd["math_score"]>=70)*100
combined_pd["% Pass Reading"] = (combined_pd["reading_score"]>=70)*100


#Isolate columns
combined_pd = combined_pd[['school_name', 'type', 'size', 'budget','reading_score', 'math_score',
       '% Pass Math', '% Pass Reading']]

#Groupby School Name and take mean
school_summary_avg = combined_pd.groupby("school_name").mean().reset_index()


#Add "Per Student Budget" Column
school_summary_avg["Per Student Budget"] = school_summary_avg["budget"]/school_summary_avg["size"]

schools_pd = schools_pd.drop(['School ID','size','budget'],axis = 1)
school_summary_avg = pd.merge(school_summary_avg,schools_pd, on ='school_name')
school_summary_avg

#Rename Columns
school_summary = school_summary_avg.rename(columns = {"school_name": "School Name",
                                                      "size": "Total Students",
                                                      "budget": "Total School Budget",
                                                      "reading_score": "Average Reading Score",
                                                      "math_score":"Average Math Score",
                                                      "type": "School Type"
                                                 })

school_summary ["Overall Pass %"] = (school_summary ["% Pass Math"]+school_summary ["% Pass Reading"])/2

school_summary = school_summary[["School Name","Total Students","Total School Budget","School Type","Per Student Budget",
                                 "Average Reading Score","Average Math Score","% Pass Reading","% Pass Math","Overall Pass %"]]
school_summary.set_index("School Name")
school_summary

,School Name,Total Students,Total School Budget,School Type,Per Student Budget,Average Reading Score,Average Math Score,% Pass Reading,% Pass Math,Overall Pass %
0,Bailey High School,4976.0,3124928.0,District,628.0,81.033963,77.048432,81.933280,66.680064,74.306672
1,Cabrera High School,1858.0,1081356.0,Charter,582.0,83.975780,83.061895,97.039828,94.133477,95.586652
2,Figueroa High School,2949.0,1884411.0,District,639.0,81.158020,76.711767,80.739234,65.988471,73.363852
3,Ford High School,2739.0,1763916.0,District,644.0,80.746258,77.102592,79.299014,68.309602,73.804308
4,Griffin High School,1468.0,917500.0,Charter,625.0,83.816757,83.351499,97.138965,93.392371,95.265668
5,Hernandez High School,4635.0,3022020.0,District,652.0,80.934412,77.289752,80.862999,66.752967,73.807983
6,Holden High School,427.0,248087.0,Charter,581.0,83.814988,83.803279,96.252927,92.505855,94.379391
7,Huang High School,2917.0,1910635.0,District,655.0,81.182722,76.629414,81.316421,65.683922,73.500171
8,Johnson High School,4761.0,3094650.0,District,650.0,80.966394,77.072464,81.222432,66.057551,73.639992
9,Pena High School,962.0,585858.0,Charter,609.0,84.044699,83.839917,95.945946,94.594595,95.270270


In [7]:
#Top Performing Schools
top_schools = school_summary.sort_values("Overall Pass %", ascending = False)
top_schools = top_schools.set_index("School Name")
top_schools.head()

,Total Students,Total School Budget,School Type,Per Student Budget,Average Reading Score,Average Math Score,% Pass Reading,% Pass Math,Overall Pass %
School Name,,,,,,,,,
Cabrera High School,1858.0,1081356.0,Charter,582.0,83.975780,83.061895,97.039828,94.133477,95.586652
Thomas High School,1635.0,1043130.0,Charter,638.0,83.848930,83.418349,97.308869,93.272171,95.290520
Pena High School,962.0,585858.0,Charter,609.0,84.044699,83.839917,95.945946,94.594595,95.270270
Griffin High School,1468.0,917500.0,Charter,625.0,83.816757,83.351499,97.138965,93.392371,95.265668
Wilson High School,2283.0,1319574.0,Charter,578.0,83.989488,83.274201,96.539641,93.867718,95.203679


In [9]:
#Bottom Performing Schools
bottom_schools = top_schools.tail()
bottom_schools

,Total Students,Total School Budget,School Type,Per Student Budget,Average Reading Score,Average Math Score,% Pass Reading,% Pass Math,Overall Pass %
School Name,,,,,,,,,
Ford High School,2739.0,1763916.0,District,644.0,80.746258,77.102592,79.299014,68.309602,73.804308
Johnson High School,4761.0,3094650.0,District,650.0,80.966394,77.072464,81.222432,66.057551,73.639992
Huang High School,2917.0,1910635.0,District,655.0,81.182722,76.629414,81.316421,65.683922,73.500171
Figueroa High School,2949.0,1884411.0,District,639.0,81.158020,76.711767,80.739234,65.988471,73.363852
Rodriguez High School,3999.0,2547363.0,District,637.0,80.744686,76.842711,80.220055,66.366592,73.293323


In [10]:
#Math and Reading Scores by Grade
avg_score_grade = students_pd.loc[:,["school_name","grade","reading_score","math_score"]]
avg_score_grade = avg_score_grade.rename(columns = {"school_name":"School",
                                         "grade": "Grade",
                                         "reading_score": "Average Reading Score",
                                         "math_score": "Average Math Score"})
avg_score_grade = avg_score_grade.groupby(["School","Grade"]).mean()
avg_score_grade.sort_values(["Grade"],ascending = False, inplace=False)
avg_score_grade

Average Reading Score  Average Math Score
School                Grade                                           
Bailey High School    10th               80.907183           76.996772
                      11th               80.945643           77.515588
                      12th               80.912451           76.492218
                      9th                81.303155           77.083676
Cabrera High School   10th               84.253219           83.154506
                      11th               83.788382           82.765560
                      12th               84.287958           83.277487
                      9th                83.676136           83.094697
Figueroa High School  10th               81.408912           76.539974
                      11th               80.640339           76.884344
                      12th               81.384863           77.151369
                      9th                81.198598           76.403037
Ford High School      10th               81.262712           77.672316
                      11th               80.403642           76.918058
                      12th               80.662338           76.179963
                      9th                80.632653           77.361345
Griffin High School   10th               83.706897           84.229064
                      11th               84.288089           83.842105
                      12th               84.013699           83.356164
                      9th                83.369193           82.044010
Hernandez High School 10th               80.660147           77.337408
                      11th               81.396140           77.136029
                      12th               80.857143           77.186567
                      9th                80.866860           77.438495
Holden High School    10th               83.324561           83.429825
                      11th               83.815534           85.000000
                      12th               84.698795           82.855422
                      9th                83.677165           83.787402
Huang High School     10th               81.512386           75.908735
                      11th               81.417476           76.446602
                      12th               80.305983           77.225641
                      9th                81.290284           77.027251
Johnson High School   10th               80.773431           76.691117
                      11th               80.616027           77.491653
                      12th               81.227564           76.863248
                      9th                81.260714           77.187857
Pena High School      10th               83.612000           83.372000
                      11th               84.335938           84.328125
                      12th               84.591160           84.121547
                      9th                83.807273           83.625455
Rodriguez High School 10th               80.629808           76.612500
                      11th               80.864811           76.395626
                      12th               80.376426           77.690748
                      9th                80.993127           76.859966
Shelton High School   10th               83.441964           82.917411
                      11th               84.373786           83.383495
                      12th               82.781671           83.778976
                      9th                84.122642           83.420755
Thomas High School    10th               84.254157           83.087886
                      11th               83.585542           83.498795
                      12th               83.831361           83.497041
                      9th                83.728850           83.590022
Wilson High School    10th               84.021452           83.724422
                      11th               83.764608           83.195326
                      12th               84.317673   

In [11]:
#Sort by School Spending
top_schools = top_schools.sort_values("Per Student Budget",ascending = False)
top_schools

bins = [575,600,625,650,675]
budget_labels = ["Below Average","Average","High","Very High"]
pd.cut(top_schools["Per Student Budget"],bins,labels = budget_labels)
top_schools ["Level of School Spending"] = pd.cut(top_schools["Per Student Budget"],bins,labels = budget_labels)

school_spending = top_schools [['Level of School Spending','Level of School Spending',
                                'Overall Pass %', 'Average Reading Score',
                                'Average Math Score', '% Pass Math', '% Pass Reading' 
                               ]]
school_spending

,Level of School Spending,Level of School Spending,Overall Pass %,Average Reading Score,Average Math Score,% Pass Math,% Pass Reading
School Name,,,,,,,
Huang High School,Very High,Very High,73.500171,81.182722,76.629414,65.683922,81.316421
Hernandez High School,Very High,Very High,73.807983,80.934412,77.289752,66.752967,80.862999
Johnson High School,High,High,73.639992,80.966394,77.072464,66.057551,81.222432
Ford High School,High,High,73.804308,80.746258,77.102592,68.309602,79.299014
Figueroa High School,High,High,73.363852,81.158020,76.711767,65.988471,80.739234
Thomas High School,High,High,95.290520,83.848930,83.418349,93.272171,97.308869
Rodriguez High School,High,High,73.293323,80.744686,76.842711,66.366592,80.220055
Bailey High School,High,High,74.306672,81.033963,77.048432,66.680064,81.933280
Griffin High School,Average,Average,95.265668,83.816757,83.351499,93.392371,97.138965


In [12]:
#Sort by School Size
sort_school_size = top_schools [['Total Students','Overall Pass %', 'Average Reading Score',
       '% Pass Reading', 'Average Math Score', '% Pass Math']]

sort_school_size = sort_school_size.sort_values("Total Students",ascending = False)
sort_school_size

,Total Students,Overall Pass %,Average Reading Score,% Pass Reading,Average Math Score,% Pass Math
School Name,,,,,,
Bailey High School,4976.0,74.306672,81.033963,81.933280,77.048432,66.680064
Johnson High School,4761.0,73.639992,80.966394,81.222432,77.072464,66.057551
Hernandez High School,4635.0,73.807983,80.934412,80.862999,77.289752,66.752967
Rodriguez High School,3999.0,73.293323,80.744686,80.220055,76.842711,66.366592
Figueroa High School,2949.0,73.363852,81.158020,80.739234,76.711767,65.988471
Huang High School,2917.0,73.500171,81.182722,81.316421,76.629414,65.683922
Ford High School,2739.0,73.804308,80.746258,79.299014,77.102592,68.309602
Wilson High School,2283.0,95.203679,83.989488,96.539641,83.274201,93.867718
Cabrera High School,1858.0,95.586652,83.975780,97.039828,83.061895,94.133477


In [13]:
#Sort by School Type
sort_type_schools = top_schools[['School Type', 'Overall Pass %', 'Average Reading Score',
       '% Pass Reading', 'Average Math Score', '% Pass Math']]

sort_type_schools = top_schools.sort_values("School Type",ascending = True)
sort_type_schools

,Total Students,Total School Budget,School Type,Per Student Budget,Average Reading Score,Average Math Score,% Pass Reading,% Pass Math,Overall Pass %,Level of School Spending
School Name,,,,,,,,,,
Thomas High School,1635.0,1043130.0,Charter,638.0,83.848930,83.418349,97.308869,93.272171,95.290520,High
Griffin High School,1468.0,917500.0,Charter,625.0,83.816757,83.351499,97.138965,93.392371,95.265668,Average
Pena High School,962.0,585858.0,Charter,609.0,84.044699,83.839917,95.945946,94.594595,95.270270,Average
Shelton High School,1761.0,1056600.0,Charter,600.0,83.725724,83.359455,95.854628,93.867121,94.860875,Below Average
Wright High School,1800.0,1049400.0,Charter,583.0,83.955000,83.682222,96.611111,93.333333,94.972222,Below Average
Cabrera High School,1858.0,1081356.0,Charter,582.0,83.975780,83.061895,97.039828,94.133477,95.586652,Below Average
Holden High School,427.0,248087.0,Charter,581.0,83.814988,83.803279,96.252927,92.505855,94.379391,Below Average
Wilson High School,2283.0,1319574.0,Charter,578.0,83.989488,83.274201,96.539641,93.867718,95.203679,Below Average
Huang High School,2917.0,1910635.0,District,655.0,81.182722,76.629414,81.316421,65.683922,73.500171,Very High
